In [ ]:
# Current directory
import os
os.chdir('E:\work\mine')   #   改变当前工作目录到指定路径

### 导入数据

In [ ]:
import numpy as np
import pandas as pd

X_new=pd.read_csv(r"extract\All_feature\features\Group5.csv",header=None)
y_new= pd.read_csv(r'extract\label.csv',header=None)

# X_new= X_new[:,1:]
print(X_new.shape)
print(y_new.shape)
X_new = np.array(X_new)
y_new = np.array(y_new).ravel()

In [ ]:
from sklearn.preprocessing import StandardScaler

# 使用 StandardScaler 进行标准化
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)

### 寻找最佳参数

In [ ]:
import statistics       # 提供数学统计的函数和工具，如平均数、中位数、众数
from sklearn.metrics import balanced_accuracy_score

# dataset splitting   将数据集分割成几个较小的独立的部分，包括训练集、验证集、测试集，保证模型在不同的数据子集上都有良好的表现
from sklearn.model_selection import train_test_split
#   20%的数据作为测试集，80%作为训练集
X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=1111)

print(X_train_whole.shape)
print(X_ind_test.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


# 定义参数范围
param_grid = {'C': np.logspace(-3, 3, num=100), 'max_iter': [5000]}

# 初始化逻辑回归模型
clf = LogisticRegression()

# 初始化网格搜索
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=10, scoring='balanced_accuracy')

# 在训练数据上进行网格搜索
grid_search.fit(X_train_whole, y_train_whole)

# 输出最优参数组合
print("Best parameters:", grid_search.best_params_)

# 输出最优参数组合下的平均值和标准差
print("Best mean balanced accuracy:", round(grid_search.best_score_, 3))
print("Standard deviation of balanced accuracy:", round(grid_search.cv_results_['std_test_score'][grid_search.best_index_], 3))


### 测试

In [ ]:
import statistics
from sklearn.metrics import confusion_matrix
import math
from sklearn.metrics import roc_auc_score


### 10折交叉验证

In [ ]:
# 10 k-fold cross vaild
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=1111)

BACC_collecton = []
ACC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []
AP=[]

y_true_all = []
y_pred_all = []
y_pred_proba_all = []

mean_recall = np.linspace(0, 1, 100)        
all_precision = []      

base_fpr = np.linspace(0, 1, 100)
mean_tpr = 0.0         

interp_tpr_collection = []      

def categorical_probas_to_classes(p):      
    return np.argmax(p, axis=1)       
# model
clf = LogisticRegression(C=0.014174741629268055 , max_iter=5000)  
skf = StratifiedKFold(n_splits=10)
for train, test in skf.split(X_train_whole, y_train_whole):
    
    X_train, X_valid, y_train, y_valid = np.take(X_train_whole, train.tolist(), axis=0), np.take(X_train_whole,
                                                                                                    test.tolist(),
                                                                                                    axis=0), np.take(
        y_train_whole, train.tolist(), axis=0), np.take(y_train_whole, test.tolist(), axis=0)
    clf.fit(X_train, y_train)

    y_pred_proba = clf.predict_proba(X_valid)

    y_valid_pred = categorical_probas_to_classes(y_pred_proba)


    y_true_all.extend(y_valid.ravel().astype(int))
    y_pred_all.extend(y_valid_pred.astype(int))
    y_pred_proba_all.extend(y_pred_proba[:,1])

    TP, FP, FN, TN = confusion_matrix(y_valid, y_valid_pred).ravel()
    Sn_collection.append(TP/(TP+FN))
    Sp_collection.append(TN/(TN+FP))
    ACC_collection.append((TP+TN)/(TP+TN+FP+FN))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collection.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))

    auc = roc_auc_score(y_valid, y_pred_proba[:, 1])
    AUC_collection.append(auc)


results_df = pd.DataFrame({
    'Predicted_Proba': np.round(y_pred_proba_all, 8),
    'True_Label': y_true_all,
    'Predicted_Label': y_pred_all
})

# Save the results to a CSV file
# results_df.to_csv(r'Result/LR/10折/LR_ESM.csv', index=False)  

# 输出结果
print(round(statistics.mean(ACC_collection),3),'±',round(statistics.stdev(ACC_collection),3))    
print(round(statistics.mean(Sn_collection),3),'±',round(statistics.stdev(Sn_collection),3))
print(round(statistics.mean(Sp_collection),3),'±',round(statistics.stdev(Sp_collection),3))
print(round(statistics.mean(MCC_collection),3),'±',round(statistics.stdev(MCC_collection),3))
print(round(statistics.mean(AUC_collection),3),'±',round(statistics.stdev(AUC_collection),3))



### 独立测试

In [ ]:
# Independence test
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# result collection list
BACC_collecton = []
ACC_collection = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
AP=[]
mean_recall = np.linspace(0, 1, 100)
all_precision = []
base_fpr = np.linspace(0, 1, 100)
mean_tpr = 0.0
# 新的TPR集合
interp_tpr_collection = []

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

for i in range(10):
    # dataset splitting
    X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=i)
    clf = LogisticRegression(C = 17.47528400007683, max_iter=5000)
    clf.fit(X_train_whole, y_train_whole)   
    y_pred_score = clf.predict_proba(X_ind_test)
    y_pred = categorical_probas_to_classes(y_pred_score)    
    y_true = y_ind_test                 
    TP, FP, FN, TN = confusion_matrix(y_true, y_pred).ravel() 
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    ACC_collection.append((TP+TN)/(TP+TN+FP+FN))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    auc = roc_auc_score(y_true, y_pred_score[:, 1])
    AUC_collecton.append(auc)
  
    
print(round(statistics.mean(ACC_collection),3),'±',round(statistics.stdev(ACC_collection),3))
print(round(statistics.mean(Sn_collecton),3),'±',round(statistics.stdev(Sn_collecton),3))
print(round(statistics.mean(Sp_collecton),3),'±',round(statistics.stdev(Sp_collecton),3))
print(round(statistics.mean(MCC_collecton),3),'±',round(statistics.stdev(MCC_collecton),3))
print(round(statistics.mean(AUC_collecton),3),'±',round(statistics.stdev(AUC_collecton),3))

